# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# 1. Загрузка и подготовка данных

##### Импорт библиотек и ознакомление с данными.

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from scipy import stats as st
import numpy as np
pd.set_option('display.max_rows', 300)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
region1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/geo_data_0.csv')
region2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/geo_data_1.csv')
region3 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/geo_data_2.csv')

In [4]:
print(region1.isna().sum())
print(region2.isna().sum())
print(region3.isna().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


In [5]:
print(region1.duplicated().sum())
print(region2.duplicated().sum())
print(region3.duplicated().sum())

0
0
0


##### Вывод: пропусков нет, данные приведены к нужным типам и готовы для подготовки моделей.

##### Удалю столбцы id.

In [6]:
region1 = region1.drop('id',axis=1)
region2 = region2.drop('id',axis=1)
region3 = region3.drop('id',axis=1)

##### Разобью выборки на features и target.

In [7]:
features_region1 = region1.drop('product',axis=1).copy()
target_region1 = region1['product'].copy()
features_region2 = region2.drop('product',axis=1).copy()
target_region2 = region2['product'].copy()
features_region3 = region3.drop('product',axis=1).copy()
target_region3 = region3['product'].copy()

##### Разобью датасеты на обучающие и валидационные.

In [8]:
features_train_region1, features_valid_region1, target_train_region1, target_valid_region1 = train_test_split(
    features_region1, target_region1, test_size=0.25, random_state=12345)
features_train_region2, features_valid_region2, target_train_region2, target_valid_region2 = train_test_split(
    features_region2, target_region2, test_size=0.25, random_state=12345)
features_train_region3, features_valid_region3, target_train_region3, target_valid_region3 = train_test_split(
    features_region3, target_region3, test_size=0.25, random_state=12345)

##### Масштабирую все числовые данные.

In [9]:
scaler = StandardScaler()
cols=['f0','f1','f2']
features_train_region1[cols] = scaler.fit_transform(features_train_region1[cols])
features_valid_region1[cols] = scaler.transform(features_valid_region1[cols])
features_train_region2[cols] = scaler.fit_transform(features_train_region2[cols])
features_valid_region2[cols] = scaler.transform(features_valid_region2[cols])
features_train_region3[cols] = scaler.fit_transform(features_train_region3[cols])
features_valid_region3[cols] = scaler.transform(features_valid_region3[cols])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3072: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.iloc._setitem_with_indexer((slice(None), indexer), value)
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3037: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

##### Данные загружены. Предобработка проведена.

# 2. Обучение и проверка модели

##### Создание функции вывода результатов обучения.

In [10]:
#def total(model_name, features_train, target_train, features_valid, target_valid):
#    model_name.fit(features_train, target_train)
#    answers = model_name.predict(features_valid)
#    rmse = round((mean_squared_error(target_valid, answers)**0.5),4)
#    predicted_mean = round(answers.mean(),4)
#    return rmse, predicted_mean

##### Обучение моделей и предсказания по выборкам.

In [11]:
#model = LinearRegression()
#rmse1, mean1 = total(model,features_train_region1,target_train_region1,features_valid_region1,target_valid_region1)
#print('Средний предсказанный запас сырья, первый регион:', mean1)
#print('RMSE модели, первый регион:', rmse1)
#rmse2, mean2 = total(model,features_train_region2,target_train_region2,features_valid_region2,target_valid_region2)
#print('Средний предсказанный запас сырья, второй регион:', mean2)
#print('RMSE модели, второй регион:', rmse2)
#rmse3, mean3 = total(model,features_train_region3,target_train_region3,features_valid_region3,target_valid_region3)
#print('Средний предсказанный запас сырья, третий регион:', mean3)
#print('RMSE модели, третий регион:', rmse3)

##### Обучение моделей, проведение предсказаний на валидационных выборках, вывод показателя RMSE.

In [12]:
model_1 = LinearRegression().fit(features_train_region1,target_train_region1)
answers_1 = model_1.predict(features_valid_region1)
rmse_region1 = mean_squared_error(target_valid_region1, answers_1)
print('Средний предсказанный запас сырья, первый регион:', round(answers_1.mean(),4))
print('RMSE модели, первый регион:', round(rmse_region1**0.5,4))
model_2 = LinearRegression().fit(features_train_region2,target_train_region2)
answers_2 = model_2.predict(features_valid_region2)
rmse_region2 = mean_squared_error(target_valid_region2, answers_2)
print('Средний предсказанный запас сырья, второй регион:', round(answers_2.mean(),4))
print('RMSE модели, второй регион:', round(rmse_region2**0.5,4))
model_3 = LinearRegression().fit(features_train_region3,target_train_region3)
answers_3 = model_3.predict(features_valid_region3)
rmse_region3 = mean_squared_error(target_valid_region3, answers_3)
print('Средний предсказанный запас сырья, третий регион:', round(answers_3.mean(),4))
print('RMSE модели, третий регион:', round(rmse_region3**0.5,4))

Средний предсказанный запас сырья, первый регион: 92.5926
RMSE модели, первый регион: 37.5794
Средний предсказанный запас сырья, второй регион: 68.7285
RMSE модели, второй регион: 0.8931
Средний предсказанный запас сырья, третий регион: 94.965
RMSE модели, третий регион: 40.0297


##### Результат:
* Модель осуществила предсказания запасов нефти по регионам.  
* Для второго региона RMSE меньше 1 тыс баррелей, на мой взгляд это хороший показатель.
* При этом средний запас по скважине для данного региона является минимальным, по сравнению с двумя другими.

# 3. Подготовка к расчёту прибыли

##### Бюджет на 1 регион в переменной budget. Стоимость 1000 баррелей нефти в переменной oil_cost.

In [13]:
budget = 10000000000
oil_cost = 450000

##### Расчёт бюджета на 1 скважину, с учетом планов запуска 200 скважин в каждом регионе.

In [14]:
budget_per_well = int(budget / 200)
budget_per_well

50000000

##### Расчёт необходимого количества сырья в одной скважине только для покрытия убытков на открытие 200 скважин в регионе.

In [15]:
needed_oil_per_well = budget_per_well / oil_cost
needed_oil_per_well

111.11111111111111

##### Расчёт необходимого запаса нефти в 200 лучших скважинах в регионе.

In [16]:
needed_oil_per_region = budget / oil_cost
needed_oil_per_region

22222.222222222223

##### Промежуточные оценки:
1. Для покрытия всех убытков на запуск 200 скважин в регионе в каждой из них должно быть запасов сырья минимум на 111 тыс. баррелей в каждой.
    * Средний предсказанный запас по скважинам на первый регион - 92.5926 тыс баррелей.
    * Средний предсказанный запас по скважинам на второй регион - 68.7285 тыс баррелей.
    * Средний предсказанный запас по скважинам на третий регион - 94.9650 тыс баррелей.  
    
Как видно из предсказаний модели, средний запас можно считать явно недостаточным для каждого региона. Остаётся надеяться на большой разброс показателей запасов между самыми крупными и самыми мелкими скважинами.
2. Для безубыточного запуска 2 каждом регионе необходим совокупный запас сырья минимум в 22222 тыс. баррелей на 200 самых крупных скважин.

# 4. Расчёт прибыли и рисков 

##### Выбор скважин с максимальными значениями предсказаний.

##### Присвоение индекса переменным с предсказаниями.

In [17]:
answers_1 = pd.Series(answers_1, index = target_valid_region1.index).copy()
answers_2 = pd.Series(answers_2, index = target_valid_region2.index).copy()
answers_3 = pd.Series(answers_3, index = target_valid_region3.index).copy()

##### Функция revenue с вычислением прибыли по 200 лучшим скважинам из выборки.

In [18]:
def revenue(targs, preds):
    preds_sorted = preds.sort_values(ascending=False)
    selected = targs[preds_sorted.index][:200]
    profit = (450000 * selected.sum()) - budget
    return profit

##### Функция bootstrap с выводом 1000 выборок по 500 скважин с вычислением прибыли с помощью функции revenue.

In [19]:
def bootstrap(targets, predictions):
    state = np.random.RandomState(12345)
    values = []
    count = 0
    for i in range(1000):
        target_subsample = targets.sample(n=500, replace=True, random_state=state)
        preds_subsample = predictions[target_subsample.index]
        income = revenue(target_subsample, preds_subsample)
        values.append(income)
        if income < 0:
            count += 1
    possibility_of_loss = round(count/1000,6)
    values = pd.Series(values)
    confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(),values.sem())
    return round(values.mean(), 2), confidence_interval, possibility_of_loss

##### Применение функции bootstrap для каждого региона.

In [20]:
values_region1, confidence_interval_region1, loss_risk1 = bootstrap(target_valid_region1,answers_1)
print('Первый регион:','\n','Доверительный интервал -',confidence_interval_region1,'\n','Валовая прибыль -', values_region1,'\n','Риск убытков -', loss_risk1,'\n')
values_region2, confidence_interval_region2, loss_risk2 = bootstrap(target_valid_region2,answers_2)
print('Второй регион:','\n','Доверительный интервал -',confidence_interval_region2,'\n','Валовая прибыль -', values_region2,'\n','Риск убытков -', loss_risk2,'\n')
values_region3, confidence_interval_region3, loss_risk3 = bootstrap(target_valid_region3,answers_3)
print('Третий регион:','\n','Доверительный интервал -',confidence_interval_region3,'\n','Валовая прибыль -', values_region3,'\n','Риск убытков -', loss_risk3,'\n')

Первый регион: 
 Доверительный интервал - (408732207.0686982, 443144846.7524858) 
 Валовая прибыль - 425938526.91 
 Риск убытков - 0.06 

Второй регион: 
 Доверительный интервал - (501621476.1763219, 528824070.7102582) 
 Валовая прибыль - 515222773.44 
 Риск убытков - 0.01 

Третий регион: 
 Доверительный интервал - (417453552.04135805, 452563173.52415365) 
 Валовая прибыль - 435008362.78 
 Риск убытков - 0.064 



### Результаты исследования:
1. Были заданы следущие параметры: 
    - Бюджет запуска скважин на 1 регион - 10 млрд. руб.
    - Выручка с 1 тыс. баррелей нефти - 450 тыс. руб.
    - Применена техника bootstrap с 1000 итераций по 500 скважин для получения репрезентативной выборки скважин.
    - Для оценки выручки и валовой прибыли по каждому региону использовались по 200 скважин с самыми лучшими показателями.
    - Исследованы три региона с известными данными о запасах нефти.
2. В результате проверки всех параметров, удовлетворяет условиях второй регион с прогнозируемой валовой прибылью 515222773.44 рублей при риске убытков около 1%.
3. По остальным регионам прогнозируется уровень риска убытка более 2.5%.